In [ ]:
#task 5 S11, onda sinusoidale con omega circa omega risonanza 4000hz

import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np

# -[Configurazione Analog Discovery 2]-----------------------------------------
#   1. Connessiene con AD2
ad2 = tdwf.AD2()



ad2.vdd = +5
ad2.vss = -5
ad2.power(True)


#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)

index = 0
a = np.linspace(0, 5, 50)
offset = False
#xy = True

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=3e6 #3e6
scope.npt=8192
scope.ch1.rng = 5
scope.ch2.rng = 5
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1)

# -[Funzioni di gestione eventi]-----------------------------------------------
def on_close(event):
    global flag_run
    flag_run = False
def on_key(event):
    global flag_run
    global flag_acq
    if event.key == 'x':  # => export su file
        filename = input("Esporta dati su file: ")
        #data = np.column_stack((scope.time.vals, scope.ch1.vals, scope.ch2.vals))
        #data = np.column_stack((tt, ch1, ch2))
        if scope.npt > 8192:
            info =  f"Acquisizione Analog Discovery 2 - Lunga durata\ntime\tch1\tch2"
        else:
            info =  f"Acquisizione Analog Discovery 2\nTimestamp {scope.time.t0}\ntime\tch1\tch2"
        #np.savetxt(filename, data, delimiter='\t', header=info)
        np.savetxt("../Data/"+filename+".txt", np.c_[tt, ch1, ch2])
    if event.key == ' ':  # => run/pausa misura
        flag_acq = not flag_acq
    if event.key == 'escape':  # => esci dalla misura
        flag_run = False

# -[Ciclo di misura]-----------------------------------------------------------
fig, axs = plt.subplots(1,2,figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('key_press_event', on_key)
flag_run = True
flag_acq = True
flag_first = True

tt = []
ch1 = []
ch2 = []


while flag_run:
    if flag_acq: # l'acquisizione è attiva?
        scope.sample()
        tempTT = scope.time.vals
        tempCH1 = scope.ch1.vals
        tempCH2 = scope.ch2.vals

        tt.append(tempTT)
        ch1.append(tempCH1)
        ch2.append(tempCH2)
    # Visualizzazione
    if flag_first:
        flag_first = False
        hp1, = plt.plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        hp2, = plt.plot(scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
        plt.legend()
        plt.grid(True)
        plt.xlabel("Time [msec]", fontsize=15)
        plt.ylabel("Signal [V]", fontsize=15)
        plt.title("User interaction: x|space|escape")
        plt.tight_layout()
        plt.show(block = False)
        
        
#        wgen.w1.config(func = tdwf.funcDC)
#        wgen.w1.offs = -1.5
        wgen.w1.config(func = tdwf.funcSine, duty=50, freq = 4e2, ampl = 0)
        if offset:
            wgen.w1.offs = -1
        else:
            wgen.w1.offs = 0

        wgen.w1.start()
    else:

        index += 1
        if index >= a.size:
            index = a.size - 1
        
        wgen.w1.ampl = a[index]

        
        

        axs[0].clear()
        axs[1].clear()

        
        #ax.plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        axs[1].plot(scope.ch1.vals, scope.ch2.vals, "-", label="Ch2(y)vsCh1(x)", color="tab:blue")
        axs[1].set_xlabel("Ch1 [V]", fontsize=15)
        axs[1].set_ylabel("Ch2 [V]", fontsize=15)

        #axs[0].plot(tt, ch1, "-", label="Ch1", color="tab:orange")
        #axs[0].plot(tt, ch2, "-", label="Ch2", color="tab:blue")
        axs[0].plot(tempTT, tempCH1, "-", label="Ch1", color="tab:orange")
        axs[0].plot(tempTT, tempCH2, "-", label="Ch2", color="tab:blue")
        axs[0].set_xlabel("Time [msec]", fontsize=15)
        axs[0].set_ylabel("Signal [V]", fontsize=15)
        #axs[0].plot(scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
        #axs[0].plot(scope.time.vals, scope.ch2.vals, "-", label="Ch2(y)", color="tab:blue")

        axs[0].set_title("User interaction: x|space|escape")
        axs[1].annotate(f"Ampl: {wgen.w1.ampl:.2f}", (0.5,0.5), xycoords='axes fraction', fontsize=15)
        axs[0].legend()
        axs[1].legend()
        axs[0].grid(True)
        axs[1].grid(True)

        fig.canvas.draw()
        fig.canvas.flush_events()

plt.close(fig)
ad2.close()

Dispositivo #1 [SN:210321AD14E6, hdwf=1] connesso!
Configurazione #1
Dispositivo disconnesso.


In [5]:
ad2.close()

Dispositivo disconnesso.
